In [22]:
import numpy as np
import pandas as pd
import datetime as dt

In [23]:
#importing the data

prices = pd.read_csv("dataairbnb_price.csv")
xls = pd.ExcelFile("dataairbnb_room_type.xlsx")
room_types = xls.parse(0)
reviews = pd.read_csv("dataairbnb_last_review.tsv", sep="\t")

In [24]:
#cleaning price column

prices["price"] = prices["price"].str.replace('dollars',' ')
prices["price"] = pd.to_numeric(prices['price'])

In [25]:
#Subset prices for listings costing $0, free_listings

free_listings = prices["price"] == 0

In [26]:
#Update prices by removing all free listings from prices

prices = prices.loc[~free_listings]

In [27]:
prices.head()

,listing_id,price,nbhood_full
0,2595,225,"Manhattan, Midtown"
1,3831,89,"Brooklyn, Clinton Hill"
2,5099,200,"Manhattan, Murray Hill"
3,5178,79,"Manhattan, Hell's Kitchen"
4,5238,150,"Manhattan, Chinatown"


In [28]:
avg_price = round(prices["price"].mean(), 2)

In [29]:
print(avg_price)

141.82


In [30]:
prices.describe()

,listing_id,price
count,2.520200e+04,25202.000000
mean,2.068920e+07,141.817316
std,1.103081e+07,147.350650
min,2.595000e+03,10.000000
25%,1.202115e+07,69.000000
50%,2.234547e+07,105.000000
75%,3.037697e+07,175.000000
max,3.645581e+07,7500.000000


In [33]:
#Add a new column to the prices DataFrame, price_per_month

prices["price_per_month"] = prices["price"] * 365 / 12

In [34]:
# average_price_per_month

average_price_per_month = round(prices["price_per_month"].mean(), 2)
difference = round((average_price_per_month - 3100),2)

In [35]:
print(average_price_per_month)

4313.61


In [48]:
room_types['room_type'].sample(10)

15192       PRIVATE ROOM
13934       PRIVATE ROOM
10765       PRIVATE ROOM
11171       Private room
8637        Private room
14175    Entire home/apt
1840     Entire home/apt
16173       Private room
19636    ENTIRE HOME/APT
22513       Private room
Name: room_type, dtype: object

In [49]:
# Convert the room_type column to lowercase

room_types["room_type"] = room_types["room_type"].str.lower()

In [50]:
room_types['room_type'].sample(10)

2836     entire home/apt
14299    entire home/apt
16802       private room
20573    entire home/apt
22746    entire home/apt
23721    entire home/apt
2997     entire home/apt
16131       private room
960      entire home/apt
16235       private room
Name: room_type, dtype: object

In [51]:
# Update the room_type column to category data type

room_types["room_type"] = room_types["room_type"].astype("category")


In [52]:
assert room_types["room_type"].dtype == 'category'

In [53]:
# Create the variable room_frequencies

room_frequencies = room_types["room_type"].value_counts()

In [54]:
print(room_frequencies)

entire home/apt    13266
private room       11356
shared room          587
Name: room_type, dtype: int64


In [57]:
reviews.head(10)

,listing_id,host_name,last_review
0,2595,Jennifer,May 21 2019
1,3831,LisaRoxanne,July 05 2019
2,5099,Chris,June 22 2019
3,5178,Shunichi,June 24 2019
4,5238,Ben,June 09 2019
5,5295,Lena,June 22 2019
6,5441,Kate,June 23 2019
7,5803,Laurie,June 24 2019
8,6021,Claudio,July 05 2019
9,6848,Allen & Irina,June 29 2019


In [59]:
# Change the data type of the last_review column to datetime

reviews["last_review"] = pd.to_datetime(reviews["last_review"])

In [60]:
reviews['last_review'].dt.year

0        2019
1        2019
2        2019
3        2019
4        2019
         ... 
25204    2019
25205    2019
25206    2019
25207    2019
25208    2019
Name: last_review, Length: 25209, dtype: int64

In [61]:
# Create first_reviewed, the earliest review date

first_reviewed = reviews["last_review"].dt.date.min()

In [62]:
print(first_reviewed)

2019-01-01


In [63]:
# Create last_reviewed, the most recent review date

last_reviewed = reviews["last_review"].dt.date.max()
print(last_reviewed)

2019-07-09


In [64]:
# join dataframes

rooms_and_prices = prices.merge(room_types, how="outer", on="listing_id")
airbnb_merged = rooms_and_prices.merge(reviews, how="outer", on="listing_id")


In [66]:
# Drop missing values from airbnb_merged

airbnb_merged.dropna(inplace=True)

In [67]:
airbnb_merged.duplicated().sum()

0

In [68]:
print(airbnb_merged)

       listing_id  price                 nbhood_full  price_per_month  \
0            2595  225.0          Manhattan, Midtown      6843.750000   
1            3831   89.0      Brooklyn, Clinton Hill      2707.083333   
2            5099  200.0      Manhattan, Murray Hill      6083.333333   
3            5178   79.0   Manhattan, Hell's Kitchen      2402.916667   
4            5238  150.0        Manhattan, Chinatown      4562.500000   
...           ...    ...                         ...              ...   
25197    36425863  129.0  Manhattan, Upper East Side      3923.750000   
25198    36427429   45.0            Queens, Flushing      1368.750000   
25199    36438336  235.0  Staten Island, Great Kills      7147.916667   
25200    36442252  100.0           Bronx, Mott Haven      3041.666667   
25201    36455809   30.0          Brooklyn, Bushwick       912.500000   

                                      description        room_type  \
0                           Skylit Midtown Castle  en

In [69]:
# Extract information from the nbhood_full column and store as a new column, borough

airbnb_merged["borough"] = airbnb_merged["nbhood_full"].str.partition(",")[0]

In [70]:
# Group by borough and calculate summary statistics

boroughs = airbnb_merged.groupby("borough")["price"].agg(["sum", "mean", "median", "count"])

In [71]:
print(boroughs)

                     sum        mean  median  count
borough                                            
Bronx            55156.0   79.247126    65.0    696
Brooklyn       1275250.0  122.021816    95.0  10451
Manhattan      1898417.0  184.044304   149.0  10315
Queens          320715.0   92.826339    70.0   3455
Staten Island    22974.0   86.044944    71.0    267


In [72]:
# Round boroughs to 2 decimal places, and sort by mean in descending order

boroughs = boroughs.round(2).sort_values("mean", ascending=False)
print(boroughs)

                     sum    mean  median  count
borough                                        
Manhattan      1898417.0  184.04   149.0  10315
Brooklyn       1275250.0  122.02    95.0  10451
Queens          320715.0   92.83    70.0   3455
Staten Island    22974.0   86.04    71.0    267
Bronx            55156.0   79.25    65.0    696


In [73]:
# Create labels for the price range, label_names

label_names = ["Budget", "Average", "Expensive", "Extravagant"]

In [74]:
# Create the label ranges, ranges

ranges = [0, 69, 175, 350, np.inf]


In [75]:
print(ranges)

[0, 69, 175, 350, inf]


In [77]:
# Insert new column, price_range, into DataFrame

airbnb_merged["price_range"] = pd.cut(airbnb_merged["price"], bins=ranges, labels=label_names)
print(airbnb_merged["price_range"])

0        Expensive
1          Average
2        Expensive
3          Average
4          Average
           ...    
25197      Average
25198       Budget
25199    Expensive
25200      Average
25201       Budget
Name: price_range, Length: 25184, dtype: category
Categories (4, object): ['Budget' < 'Average' < 'Expensive' < 'Extravagant']


In [78]:
# Calculate occurence frequencies for each label, prices_by_borough

prices_by_borough = airbnb_merged.groupby(["borough", "price_range"])["price_range"].count()
print(prices_by_borough)

borough        price_range
Bronx          Budget          381
               Average         285
               Expensive        25
               Extravagant       5
Brooklyn       Budget         3194
               Average        5532
               Expensive      1466
               Extravagant     259
Manhattan      Budget         1148
               Average        5285
               Expensive      3072
               Extravagant     810
Queens         Budget         1631
               Average        1505
               Expensive       291
               Extravagant      28
Staten Island  Budget          124
               Average         123
               Expensive        20
               Extravagant       0
Name: price_range, dtype: int64


In [80]:
airbnb_analysis = {'avg_price':avg_price,
            'average_price_per_month': average_price_per_month,  
            'difference':difference,          
            'room_frequencies':room_frequencies, 
            'first_reviewed': first_reviewed,
            'last_reviewed': last_reviewed,
            'prices_by_borough':prices_by_borough}

In [81]:
print(airbnb_analysis)

{'avg_price': 141.82, 'average_price_per_month': 4313.61, 'difference': 1213.61, 'room_frequencies': entire home/apt    13266
private room       11356
shared room          587
Name: room_type, dtype: int64, 'first_reviewed': datetime.date(2019, 1, 1), 'last_reviewed': datetime.date(2019, 7, 9), 'prices_by_borough': borough        price_range
Bronx          Budget          381
               Average         285
               Expensive        25
               Extravagant       5
Brooklyn       Budget         3194
               Average        5532
               Expensive      1466
               Extravagant     259
Manhattan      Budget         1148
               Average        5285
               Expensive      3072
               Extravagant     810
Queens         Budget         1631
               Average        1505
               Expensive       291
               Extravagant      28
Staten Island  Budget          124
               Average         123
               Expensive  